# Distributed Data Classification with NeMo Curator's `PromptTaskComplexityClassifier`

This notebook demonstrates the use of NeMo Curator's `PromptTaskComplexityClassifier`. The [prompt task and complexity classifier](https://huggingface.co/nvidia/prompt-task-and-complexity-classifier) a multi-headed model which classifies English text prompts across task types and complexity dimensions. It helps with data annotation, which is useful in data blending for foundation model training. Please refer to the NemoCurator Prompt Task and Complexity Classifier Hugging Face page for more information about the prompt task and complexity classifier, including its output labels, here: https://huggingface.co/nvidia/prompt-task-and-complexity-classifier.

The prompt task and complexity classifier is accelerated using [CrossFit](https://github.com/rapidsai/crossfit), a library that leverages intellegent batching and RAPIDS to accelerate the offline inference on large datasets.

Before running this notebook, please see this [Getting Started](https://github.com/NVIDIA/NeMo-Curator?tab=readme-ov-file#get-started) page for instructions on how to install NeMo Curator.

In [1]:
# Silence Warnings (HuggingFace internal warnings)

%env PYTHONWARNINGS=ignore
import warnings
warnings.filterwarnings("ignore")

env: PYTHONWARNINGS=ignore


In [2]:
from nemo_curator import get_client
from nemo_curator.classifiers import PromptTaskComplexityClassifier
from nemo_curator.datasets import DocumentDataset
import cudf
import dask_cudf

In [3]:
client = get_client(cluster_type="gpu")

cuDF Spilling is enabled


# Set Output File Path

The user should specify an empty directory below for storing the output results.

In [ ]:
output_file_path = "./prompt_task_complexity_results/"

# Prepare Text Data and Initialize Classifier

In [5]:
# Create sample DataFrame
text = ["Prompt: Write a Python script that uses a for loop."]
df = cudf.DataFrame({"text": text})
input_dataset = DocumentDataset(dask_cudf.from_cudf(df, npartitions=1))
write_to_filename = False

# Alternatively, read existing directory of JSONL files
# input_file_path="/input_data_dir/"
# input_dataset = DocumentDataset.read_json(
#     input_file_path, backend="cudf", add_filename=True
# )
# write_to_filename = True

In [6]:
classifier = PromptTaskComplexityClassifier(batch_size=1024)

# Run the  Classifier

Dask operations are lazy, so the the classifier will not run until we call an eager operation like `to_json`, `compute`, or `persist`. 

In [7]:
%%time

result_dataset = classifier(dataset=input_dataset)
result_dataset.to_json(output_path=output_file_path, write_to_filename=write_to_filename)

Starting prompt task and complexity classifier inference


GPU: tcp://127.0.0.1:34849, Part: 0: 100%|██████████| 1/1 [00:04<00:00,  4.95s/it]

Writing to disk complete for 1 partition(s)
CPU times: user 2.52 s, sys: 1.54 s, total: 4.06 s
Wall time: 20 s


GPU: tcp://127.0.0.1:34849, Part: 0: 100%|██████████| 1/1 [00:07<00:00,  7.77s/it]


# Inspect the Output

In [8]:
output_dataset = DocumentDataset.read_json(output_file_path, backend="cudf", add_filename=write_to_filename)
output_dataset.head()

Reading 1 files


,constraint_ct,contextual_knowledge,creativity_scope,domain_knowledge,no_label_reason,number_of_few_shots,prompt_complexity_score,reasoning,task_type_1,task_type_2,task_type_prob,text
0,0.5586,0.0559,0.0825,0.9803,0.0,0,0.2783,0.0632,Code Generation,Text Generation,0.767,Prompt: Write a Python script that uses a for ...
